## Data Reading, Cleaning and Improving

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('kidney_disease.csv')

In [3]:
df.tail()

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


id   age    bp     sg   al   su     rbc      pc         pcc          ba   
395  395  55.0  80.0  1.020  0.0  0.0  normal  normal  notpresent  notpresent  \
396  396  42.0  70.0  1.025  0.0  0.0  normal  normal  notpresent  notpresent   
397  397  12.0  80.0  1.020  0.0  0.0  normal  normal  notpresent  notpresent   
398  398  17.0  60.0  1.025  0.0  0.0  normal  normal  notpresent  notpresent   
399  399  58.0  80.0  1.025  0.0  0.0  normal  normal  notpresent  notpresent   

     ...  pcv    wc   rc  htn  dm  cad appet  pe ane classification  
395  ...   47  6700  4.9   no  no   no  good  no  no         notckd  
396  ...   54  7800  6.2   no  no   no  good  no  no         notckd  
397  ...   49  6600  5.4   no  no   no  good  no  no         notckd  
398  ...   51  7200  5.9   no  no   no  good  no  no         notckd  
399  ...   53  6800  6.1   no  no   no  good  no  no         notckd  

[5 rows x 26 columns]

In [4]:
columns = pd.read_csv(r"D:\GIT and Programming\Chronic Kidney Disease/data_description.txt", sep='-')
columns = columns.reset_index()

In [5]:
columns.columns=['cols','abb_col_names']

In [6]:
df.columns=columns['abb_col_names'].values

In [7]:
def convert_dtype(df, feature):
    df[feature] = pd.to_numeric(df[feature], errors='coerce')

In [8]:
features = ['packed cell volume', 'white blood cell count', 'red blood cell count']
for feature in features:
    convert_dtype(df, feature)

In [9]:
df.drop('id', axis=1, inplace=True)

In [10]:
def extract_cat_num(df):
    cat_col = [col for col in df.columns if df[col].dtype == 'object']
    num_col = [col for col in df.columns if df[col].dtype != 'object']
    return cat_col, num_col

In [11]:
cat_col, num_col = extract_cat_num(df)

In [12]:
df['diabetes mellitus'].replace(to_replace={'\tno':'no', '\tyes':'yes'}, inplace=True)
df['coronary artery disease'].replace(to_replace={'\tno':'no'}, inplace=True)
df['class'] = df['class'].replace(to_replace='ckd\t', value='ckd')

In [13]:
data = df.copy()

In [14]:
def assigning_missing_values(feature):
    random_sample=data[feature].dropna().sample(data[feature].isnull().sum())
    random_sample.index = data[data[feature].isnull()].index
    data.loc[data[feature].isnull(), feature] = random_sample

In [15]:
for col in num_col:
    assigning_missing_values(col)

In [16]:
for col in cat_col:
    assigning_missing_values(col)

In [17]:
le = LabelEncoder()

In [18]:
for col in cat_col:
    data[col] = le.fit_transform(data[col])

## Train Test Splitting

In [19]:
def data_split(entry, ratio):
    # np.random.seed(42)
    shuffled = np.random.permutation(len(entry))
    test_set_size = int(len(entry) * ratio)
    test_indices = shuffled[:test_set_size]
    train_indices = shuffled[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [20]:
train, test = data_split(data, 0.25)

In [21]:
train

age  blood pressure  specific gravity  albumin  sugar  red blood cells   
222  74.0            60.0             1.015      0.0    0.0                1  \
184  54.0            60.0             1.015      3.0    2.0                0   
96   60.0            80.0             1.010      1.0    1.0                1   
374  79.0            80.0             1.025      0.0    0.0                1   
369  75.0            70.0             1.020      0.0    0.0                1   
..    ...             ...               ...      ...    ...              ...   
234  37.0           100.0             1.010      0.0    0.0                0   
297  53.0            60.0             1.025      0.0    0.0                1   
342  44.0            60.0             1.020      0.0    0.0                1   
194  80.0            70.0             1.010      2.0    0.0                0   
202  78.0            60.0             1.020      0.0    5.0                1   

      pus cell  pus cell clumps  bacteria  blood glucose random  ...   
222          1                0         0                 108.0  ...  \
184          0                0         0                 352.0  ...   
96           1                0         0                 172.0  ...   
374          1                0         0                 111.0  ...   
369          1                0         0                 107.0  ...   
..         ...              ...       ...                   ...  ...   
234          1                0         0                 104.0  ...   
297          1                0         0                 116.0  ...   
342          1                0         0                  95.0  ...   
194          0                0         0                 169.0  ...   
202          0                0         0                 114.0  ...   

     packed cell volume  white blood cell count  red blood cell count   
222                32.0                 10200.0                   5.7  \
184                31.0                  5800.0                   3.6   
96                 36.0                  7800.0                   3.4   
374                40.0                  8000.0                   6.4   
369                46.0                 10300.0                   4.8   
..                  ...                     ...                   ...   
234                44.0                  4100.0                   5.2   
297                45.0                  7700.0                   5.2   
342                50.0                  7700.0                   6.3   
194                28.0                  8100.0                   2.9   
202                24.0                  4300.0                   3.9   

     ypertension  diabetes mellitus  coronary artery disease  appetite   
222            1                  2                        0         0  \
184            1                  2                        1         1   
96             0                  2                        1         1   
374            0                  1                        0         0   
369            0                  1                        0         0   
..           ...                ...                      ...       ...   
234            1                  1                        0         0   
297            0                  1                        0         0   
342            0                  1                        0         0   
194            1                  2                        0         0   
202            0                  2                        0         0   

     pedal edema  anemia  class  
222            0       0      0  
184            1       0      0  
96             0       0      0  
374            0       0      1  
369            0       0      1  
..           ...     ...    ...  
234            0       0      0  
297            0       0      1  
342            0       0      1  
194            0       0      0  
202            0       1      0  

[300 

In [22]:
X_train = train[['age', 'blood pressure', 'specific gravity', 'albumin', 'sugar',
       'red blood cells', ' pus cell', 'pus cell clumps', 'bacteria',
       'blood glucose random', 'blood urea', 'serum creatinine', 'sodium',
       'potassium', 'haemoglobin', 'packed cell volume',
       'white blood cell count', 'red blood cell count', 'ypertension',
       'diabetes mellitus', 'coronary artery disease', 'appetite',
       'pedal edema', 'anemia']].to_numpy()

In [23]:
X_test = test[['age', 'blood pressure', 'specific gravity', 'albumin', 'sugar',
       'red blood cells', ' pus cell', 'pus cell clumps', 'bacteria',
       'blood glucose random', 'blood urea', 'serum creatinine', 'sodium',
       'potassium', 'haemoglobin', 'packed cell volume',
       'white blood cell count', 'red blood cell count', 'ypertension',
       'diabetes mellitus', 'coronary artery disease', 'appetite',
       'pedal edema', 'anemia']].to_numpy()

In [24]:
Y_train = train[['class']].to_numpy().reshape(300,)

In [25]:
Y_test = test[['class']].to_numpy().reshape(100,)

In [26]:
Y_train

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0])

In [27]:
clf = LogisticRegression(solver='lbfgs', max_iter=10000)
clf.fit(X_train, Y_train)

LogisticRegression(max_iter=10000)

In [28]:
train

age  blood pressure  specific gravity  albumin  sugar  red blood cells   
222  74.0            60.0             1.015      0.0    0.0                1  \
184  54.0            60.0             1.015      3.0    2.0                0   
96   60.0            80.0             1.010      1.0    1.0                1   
374  79.0            80.0             1.025      0.0    0.0                1   
369  75.0            70.0             1.020      0.0    0.0                1   
..    ...             ...               ...      ...    ...              ...   
234  37.0           100.0             1.010      0.0    0.0                0   
297  53.0            60.0             1.025      0.0    0.0                1   
342  44.0            60.0             1.020      0.0    0.0                1   
194  80.0            70.0             1.010      2.0    0.0                0   
202  78.0            60.0             1.020      0.0    5.0                1   

      pus cell  pus cell clumps  bacteria  blood glucose random  ...   
222          1                0         0                 108.0  ...  \
184          0                0         0                 352.0  ...   
96           1                0         0                 172.0  ...   
374          1                0         0                 111.0  ...   
369          1                0         0                 107.0  ...   
..         ...              ...       ...                   ...  ...   
234          1                0         0                 104.0  ...   
297          1                0         0                 116.0  ...   
342          1                0         0                  95.0  ...   
194          0                0         0                 169.0  ...   
202          0                0         0                 114.0  ...   

     packed cell volume  white blood cell count  red blood cell count   
222                32.0                 10200.0                   5.7  \
184                31.0                  5800.0                   3.6   
96                 36.0                  7800.0                   3.4   
374                40.0                  8000.0                   6.4   
369                46.0                 10300.0                   4.8   
..                  ...                     ...                   ...   
234                44.0                  4100.0                   5.2   
297                45.0                  7700.0                   5.2   
342                50.0                  7700.0                   6.3   
194                28.0                  8100.0                   2.9   
202                24.0                  4300.0                   3.9   

     ypertension  diabetes mellitus  coronary artery disease  appetite   
222            1                  2                        0         0  \
184            1                  2                        1         1   
96             0                  2                        1         1   
374            0                  1                        0         0   
369            0                  1                        0         0   
..           ...                ...                      ...       ...   
234            1                  1                        0         0   
297            0                  1                        0         0   
342            0                  1                        0         0   
194            1                  2                        0         0   
202            0                  2                        0         0   

     pedal edema  anemia  class  
222            0       0      0  
184            1       0      0  
96             0       0      0  
374            0       0      1  
369            0       0      1  
..           ...     ...    ...  
234            0       0      0  
297            0       0      1  
342            0       0      1  
194            0       0      0  
202            0       1      0  

[300 

In [29]:
clf.predict_proba([[5.0,	70.0,	1.025,	1.0,	0.0,	0,	1,	0,	0,	97.0,	
             56, 3.8, 111, 2.5, 11.2,
             34.0,	7200.0,	4.1,	0,	1,	0,	1,	1, 0]])

array([[9.99726510e-01, 2.73490418e-04]])

In [30]:
input_features = [5.0,	70.0,	1.025,	1.0,	0.0,	0,	1,	0,	0,	97.0,	
             56, 3.8, 111, 2.5, 11.2,
             34.0,	7200.0,	4.1,	0,	1,	0,	1,	1, 0]
ckd_prob = clf.predict_proba([input_features])[0][1]

In [31]:
ckd_prob

0.0002734904181819465